## Imports

In [2]:
import pandas as pd
import sqlalchemy as sa
import sqlite3
import openai
import pyodbc
import pypyodbc as odbc

## Database credentials

In [3]:
''''
server='like4.database.windows.net'
database='like'
connection_string='Driver={ODBC Driver 18 for SQL Server};Server=tcp:like4.database.windows.net,1433;Database=like;Uid=CloudSA42bee827;Pwd={XP2U@X3R5EOQ23};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
conn=odbc.connect(connection_string)
'''

"'\nserver='like4.database.windows.net'\ndatabase='like'\nconnection_string='Driver={ODBC Driver 18 for SQL Server};Server=tcp:like4.database.windows.net,1433;Database=like;Uid=CloudSA42bee827;Pwd={XP2U@X3R5EOQ23};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'\nconn=odbc.connect(connection_string)\n"

In [4]:
''''
sql=''' 
#SELECT count(user_id)
#from dbo.Users 
'''
cursor=conn.cursor()
cursor.execute(sql)
dataset=cursor.fetchall()
print(dataset)
'''

'\ncursor=conn.cursor()\ncursor.execute(sql)\ndataset=cursor.fetchall()\nprint(dataset)\n'

## Connecting to the data warehouse

In [5]:
f = open("Credintials.txt", "r")
email = None
password = None
for i, line in enumerate(f):
    if i == 0:
        email = line.strip()  # Assign the first line to email variable
    elif i == 1:
        password = line.strip()  # Assign the second line to password variable

f.close()

In [6]:
server = 'anmx3c2yurjethctgyba6xtuaq-vkvbn7t4322edeguc3ufhjtjeu.datawarehouse.pbidedicated.windows.net'
database = 'likeCard'
driver='ODBC Driver 18 for SQL Server'
user= email # use %40 inplace of @
pass1=password # use %40 inplace of @   

# Create a connection string for SQL Server using Windows authentication
connection_string = f'mssql+pyodbc://{user}:{pass1}@{server}/{database}?driver={driver}&Trusted_Connection=no&Authentication=ActiveDirectoryInteractive'

In [7]:
engine = sa.create_engine(connection_string, echo=True, connect_args={'autocommit': True}, fast_executemany=True)


## Data Preprocessing

In [8]:
sql_query = '''
SELECT 
    TABLE_SCHEMA,
    TABLE_NAME,
    COLUMN_NAME,
    DATA_TYPE
FROM 
    INFORMATION_SCHEMA.COLUMNS
WHERE 
    TABLE_NAME IN (SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE')
    and TABLE_NAME like 'dim_product' 
    or TABLE_NAME like 'fact_trans%' 
    
'''
df = pd.read_sql(sql_query, engine)

DBAPIError: (pyodbc.Error) ('FA004', "[FA004] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Failed to authenticate the user 'a.talaat@likecard.com' in Active Directory (Authentication option is 'ActiveDirectoryInteractive').\r\nError code 0x4C7; state 10\r\nThe operation was canceled by the user. (0) (SQLDriverConnect); [FA004] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Failed to authenticate the user 'a.talaat@likecard.com' in Active Directory (Authentication option is 'ActiveDirectoryInteractive').\r\nError code 0x4C7; state 10\r\nThe operation was canceled by the user. (0)")
(Background on this error at: https://sqlalche.me/e/14/dbapi)

In [8]:
table_schema=df.loc[0][0]

In [9]:
df=df.drop(columns=['TABLE_SCHEMA'])


In [10]:
df.columns

Index(['TABLE_NAME', 'COLUMN_NAME', 'DATA_TYPE'], dtype='object')

In [11]:
df_dict = df.to_dict(orient='records')
df_dict

[{'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'OrderDateTime',
  'DATA_TYPE': 'datetime2'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'user_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'order_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'store_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'payment_method_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'bank_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'product_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions', 'COLUMN_NAME': 'qty', 'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'cost',
  'DATA_TYPE': 'float'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'coupon_code',
  'DATA_TYPE': 'varchar'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'fees',
  'DATA_TYPE': 'float'},
 {'TABL

In [12]:
openai.api_type = "azure"
openai.api_base = "https://like-card-test.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "85889c7998dd4adb9a4c89abe56b1242"
openai.azure_endpoint="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview"

In [13]:
text = 'what is product name that has the most purchases'

In [15]:
def generate_sql_query(table_names,text,table_schema):
    prompt = """You are a ChatGPT language model that can generate concise  SQL server queries. Please provide a natural language input text and I will generate the corresponding SQL query **without using descriptive words only sql query** in function calls or expressions for example replace DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY) with DATE_SUB(CURRENT_DATE(),  1 ) . Use only function names and parentheses. the tables names , coulmns names and datatyps in this dictionary {}  and the tables schema is {} and Input: {} SQL Query (without descriptive words)  and note that you have multible tables so you can join the needed tables as needed:""".format(df_dict, table_schema, text)
    request = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    query = request.choices[0].message.content
    return query    
query = generate_sql_query(df_dict,text,table_schema)
query


'SELECT product.name\nFROM fact_transactions\nJOIN dim_products ON fact_transactions.product_id = dim_products.id\nJOIN dim_time ON fact_transactions.OrderTimeKey = dim_time.OrderTimeKey\nGROUP BY product.name\nORDER BY SUM(fact_transactions.qty) DESC\nLIMIT 1;'

In [16]:
print(query)

SELECT product.name
FROM fact_transactions
JOIN dim_products ON fact_transactions.product_id = dim_products.id
JOIN dim_time ON fact_transactions.OrderTimeKey = dim_time.OrderTimeKey
GROUP BY product.name
ORDER BY SUM(fact_transactions.qty) DESC
LIMIT 1;


In [24]:
def executeSQLquery(table_name,text,schema_list ,max_retries):
    retry_count = 0

    while retry_count < max_retries:
        try:
            query = generate_sql_query(table_name,text,schema_list)
            result = pd.read_sql(query,engine)
            return result
        except Exception as e:
            print(f"Error executing query: {e}")
            retry_count += 1
            if retry_count < max_retries:
                print(f"Retrying... (Attempt {retry_count}/{max_retries})")
            else:
                print("Max retry attempts reached. Exiting.")
                raise

In [25]:
result = executeSQLquery(df_dict,text,table_schema,10)
result

2024-01-04 16:58:39,696 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2024-01-04 16:58:39,697 INFO sqlalchemy.engine.Engine [cached since 733.6s ago] ('BASE TABLE', 'SELECT TOP 1 product_name \nFROM fact_transactions \nJOIN dim_product \nON fact_transactions.product_id = dim_product.id \nGROUP BY product_name \nORDER BY SUM(qty) DESC', 'dbo')
2024-01-04 16:58:40,200 INFO sqlalchemy.engine.Engine SELECT TOP 1 product_name 
FROM fact_transactions 
JOIN dim_product 
ON fact_transactions.product_id = dim_product.id 
GROUP BY product_name 
ORDER BY SUM(qty) DESC
2024-01-04 16:58:40,200 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-04 16:58:40,432 INFO sqlalchemy.engine.Engine ROLLBACK
Error exe

,product_id,
0,1698,2492990


In [31]:
def get_result_prompt(question,query,df_dict,text,table_schema):

    values = executeSQLquery(df_dict,text,table_schema,10)
    prompt = """I want you to answer the input question based on the table schema ,column name , column data type,input sql query where you can join between tables and it's result, if the input result is empty then return the output as 'No data found', You should not tell about sql query and tables in the result, If something wrong then return 'Try it in another way'. tables schema are: {}, input SQL is: {}, input result is :{} and the input question is: {}. The output should be in natural language text.""".format(table_schema,query,values,question)
    return prompt
result_prompt = get_result_prompt(text,query,df_dict,text,table_schema)

2024-01-04 17:10:37,339 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2024-01-04 17:10:37,340 INFO sqlalchemy.engine.Engine [cached since 1451s ago] ('BASE TABLE', 'SELECT product_name \nFROM dim_products\nINNER JOIN fact_transactions ON dim_products.product_id = fact_transactions.product_id\nGROUP BY product_name\nORDER BY SUM(qty) DESC\nLIMIT 1;', 'dbo')
2024-01-04 17:10:37,871 INFO sqlalchemy.engine.Engine SELECT product_name 
FROM dim_products
INNER JOIN fact_transactions ON dim_products.product_id = fact_transactions.product_id
GROUP BY product_name
ORDER BY SUM(qty) DESC
LIMIT 1;
2024-01-04 17:10:37,871 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-04 17:10:38,136 INFO sqlalchemy.eng

In [1]:
request = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=[
            {"role": "user", "content": result_prompt},
        ]
    )
answer = request.choices[0].message.content
answer

NameError: name 'openai' is not defined